**CONFIGURATION**

This file holds all the configurable parameters, making it easy to tweak things without digging through the code.

In [ ]:
# This is a centralized configuration file for the project.
# All hyperparameters and paths are defined here for easy access and modification.

class Config:
    # Data Parameters for Kaggle Environment
    DATA_DIR = "/kaggle/input/fetal-head-ultrasound-dataset-for-image-segment/"
    
    # Image Dimensions
    IMG_HEIGHT = 256
    IMG_WIDTH = 256
    IMG_CHANNELS = 3

    # Training Hyperparameters
    EPOCHS = 50
    BATCH_SIZE = 8
    LEARNING_RATE = 1e-4

    # Model & Reproducibility
    RANDOM_SEED = 42
    # PyTorch models are typically saved with a .pth or .pt extension
    MODEL_SAVE_PATH = "/kaggle/working/AttentionUNet_PyTorch.pth"
    
    # Specify number of classes for the model output
    NUM_CLASSES = 1

**DATA LOADER**

This module handles all aspects of data loading, preprocessing, and splitting.

In [ ]:
# This module handles data loading and preprocessing using PyTorch's Dataset and DataLoader.

import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_data_paths_and_split(config):
    """Reads CSV, creates full paths, and splits data."""
    df = pd.read_csv(os.path.join(config.DATA_DIR, 'data.csv'))
    df['image_path'] = df['Image'].apply(lambda x: os.path.join(config.DATA_DIR, x))
    df['mask_path'] = df['Mask'].apply(lambda x: os.path.join(config.DATA_DIR, x))
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=config.RANDOM_SEED)
    train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=config.RANDOM_SEED)
    
    print(f"\nDataset Split:")
    print(f"Training samples:   {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples:       {len(test_df)}")
    
    return train_df, val_df, test_df

class FetalHeadDataset(Dataset):
    """Custom PyTorch Dataset for fetal head ultrasound images."""
    def __init__(self, dataframe, transforms=None):
        self.df = dataframe
        self.transforms = transforms
        self.image_paths = dataframe['image_path'].values
        self.mask_paths = dataframe['mask_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        
        # Load image and mask as numpy arrays
        image = np.array(Image.open(image_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        
        # Binarize mask
        mask[mask > 0] = 1.0
        
        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            # Add channel dimension to mask: (H, W) -> (1, H, W)
            mask = mask.unsqueeze(0)
            
        return image, mask

def get_loaders(config):
    """Creates and returns the training, validation, and test DataLoaders."""
    train_df, val_df, test_df = get_data_paths_and_split(config)
    
    # Define augmentations and transformations
    # ToTensorV2 handles normalization if mean/std are provided and converts to tensor
    train_transforms = A.Compose([
        A.Resize(height=config.IMG_HEIGHT, width=config.IMG_WIDTH),
        A.Rotate(limit=35, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),
        ToTensorV2(),
    ])
    
    val_test_transforms = A.Compose([
        A.Resize(height=config.IMG_HEIGHT, width=config.IMG_WIDTH),
        A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),
        ToTensorV2(),
    ])
    
    train_dataset = FetalHeadDataset(train_df, transforms=train_transforms)
    val_dataset = FetalHeadDataset(val_df, transforms=val_test_transforms)
    test_dataset = FetalHeadDataset(test_df, transforms=val_test_transforms)
    
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False)
    
    return train_loader, val_loader, test_loader


**TRAINING THE MODEL**

This module contains the core training and validation loops.

In [ ]:
# This module manages the model training and validation loops.

import torch
import torch.optim as optim
from tqdm import tqdm
from model_pytorch import criterion, dice_coeff

def train_one_epoch(loader, model, optimizer, device):
    """Runs a single training epoch."""
    model.train()
    loop = tqdm(loader, leave=True)
    running_loss = 0.0
    
    for batch_idx, (data, targets) in enumerate(loop):
        data, targets = data.to(device), targets.to(device)
        
        # Forward pass
        predictions = model(data)
        loss = criterion(predictions, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        
    return running_loss / len(loader)

def validate(loader, model, device):
    """Evaluates the model on the validation set."""
    model.eval()
    total_dice_score = 0
    total_loss = 0
    
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            predictions = model(data)
            
            # Calculate loss
            loss = criterion(predictions, targets)
            total_loss += loss.item()
            
            # Calculate metrics
            preds_sig = torch.sigmoid(predictions)
            preds_binary = (preds_sig > 0.5).float()
            total_dice_score += dice_coeff(preds_binary, targets)

    avg_loss = total_loss / len(loader)
    avg_dice = total_dice_score / len(loader)
    
    print(f"Validation -> Avg Loss: {avg_loss:.4f}, Avg Dice Score: {avg_dice:.4f}")
    return avg_loss, avg_dice

def train_model(config, model, train_loader, val_loader, device):
    """The main training function."""
    print("\n--- Starting Model Training ---")
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5, factor=0.1, verbose=True)
    
    best_val_dice = -1.0
    history = {'train_loss': [], 'val_loss': [], 'val_dice': []}

    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(train_loader, model, optimizer, device)
        val_loss, val_dice = validate(val_loader, model, device)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_dice'].append(val_dice.item())
        
        # Update learning rate scheduler
        scheduler.step(val_dice)
        
        # Save the best model
        if val_dice > best_val_dice:
            best_val_dice = val_dice
            torch.save(model.state_dict(), config.MODEL_SAVE_PATH)
            print(f"-> New best model saved with Dice Score: {val_dice:.4f}")
            
    return model, history


**EVALUATION**

This module will calculate all the final metrics on the test set.

In [ ]:
# This cell handles model evaluation on the test set and generates a results table.

import numpy as np
import pandas as pd
import torch
import cv2
from scipy.spatial.distance import directed_hausdorff
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score
from tqdm import tqdm
from model_pytorch import dice_coeff

def calculate_surface_distances(true_mask, pred_mask):
    """Calculates Hausdorff Distance and ASD."""
    # Squeeze to remove channel dim and convert to uint8
    true_mask_u8 = true_mask.squeeze().astype(np.uint8)
    pred_mask_u8 = pred_mask.squeeze().astype(np.uint8)
    
    true_contours, _ = cv2.findContours(true_mask_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    pred_contours, _ = cv2.findContours(pred_mask_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if not true_contours or not pred_contours: return np.nan, np.nan
    
    true_points = np.squeeze(true_contours[0])
    pred_points = np.squeeze(pred_contours[0])
    
    if len(true_points.shape) < 2 or len(pred_points.shape) < 2: return np.nan, np.nan
    
    hd1 = directed_hausdorff(true_points, pred_points)[0]
    hd2 = directed_hausdorff(pred_points, true_points)[0]
    hausdorff_dist = max(hd1, hd2)
    
    dists_t_to_p = np.array([cv2.pointPolygonTest(pred_contours[0], tuple(pt), True) for pt in true_points])
    dists_p_to_t = np.array([cv2.pointPolygonTest(true_contours[0], tuple(pt), True) for pt in pred_points])
    asd = (np.mean(np.abs(dists_t_to_p)) + np.mean(np.abs(dists_p_to_t))) / 2.0
    return hausdorff_dist, asd

def evaluate_model(model, loader, device):
    """Evaluates the model on a given dataset and returns a results DataFrame."""
    print("\n--- Starting Comprehensive Model Evaluation ---")
    model.eval()
    results = []
    
    with torch.no_grad():
        for data, targets in tqdm(loader, "Evaluating"):
            data, targets = data.to(device), targets.to(device)
            preds_logits = model(data)
            preds_sig = torch.sigmoid(preds_logits)
            preds_binary = (preds_sig > 0.5).float()
            
            # Move tensors to CPU and convert to numpy for sklearn/scipy metrics
            targets_np = targets.cpu().numpy()
            preds_binary_np = preds_binary.cpu().numpy()
            
            for i in range(targets_np.shape[0]):
                tm, pm = targets_np[i], preds_binary_np[i]
                tm_flat, pm_flat = tm.flatten(), pm.flatten()
                
                hd, asd = calculate_surface_distances(tm, pm)
                results.append({
                    "Accuracy": accuracy_score(tm_flat, pm_flat),
                    "Precision": precision_score(tm_flat, pm_flat, zero_division=0),
                    "Recall": recall_score(tm_flat, pm_flat, zero_division=0),
                    "F1-Score": f1_score(tm_flat, pm_flat, zero_division=0),
                    "Dice-Coefficient": dice_coeff(preds_binary[i], targets[i]).item(),
                    "IoU": jaccard_score(tm_flat, pm_flat, zero_division=0),
                    "Hausdorff-Distance": hd,
                    "ASD": asd
                })

    results_df = pd.DataFrame(results).dropna()
    return results_df

def display_results_table(results_df):
    """Prints a publication-ready table of evaluation metrics."""
    summary_stats = results_df.agg(['mean', 'std']).T
    summary_stats.columns = ['Mean', 'Standard Deviation']
    summary_stats['Mean ± SD'] = summary_stats.apply(lambda row: f"{row['Mean']:.4f} ± {row['Standard Deviation']:.4f}", axis=1)
    print("\n--- Quantitative Evaluation Results ---")
    print(summary_stats[['Mean ± SD']])


**VISUALIZATION**

This module visualizes all the data

In [ ]:
# Contains all functions for visualizing results.

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from sklearn.metrics import confusion_matrix

def plot_learning_curves(history):
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history['val_dice'], label='Validation Dice Coef')
    plt.title('Validation Dice Coefficient vs. Epochs'); plt.ylabel('Dice Coefficient'); plt.xlabel('Epoch'); plt.legend(loc='best')
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Loss vs. Epochs'); plt.ylabel('Loss'); plt.xlabel('Epoch'); plt.legend(loc='best')
    plt.show()

def visualize_predictions(model, loader, device, num_samples=5):
    model.eval()
    images, masks = next(iter(loader))
    images, masks = images.to(device), masks.to(device)
    
    with torch.no_grad():
        preds_logits = model(images)
        preds = torch.sigmoid(preds_logits) > 0.5

    # Move to CPU and convert to numpy for plotting
    images = images.cpu().numpy()
    masks = masks.cpu().numpy()
    preds = preds.cpu().numpy()

    plt.figure(figsize=(15, 5 * num_samples))
    for i in range(num_samples):
        # Transpose image from (C, H, W) to (H, W, C) for display
        img_to_show = np.transpose(images[i], (1, 2, 0))
        
        plt.subplot(num_samples, 3, i*3 + 1); plt.imshow(img_to_show); plt.title("Original Image"); plt.axis('off')
        plt.subplot(num_samples, 3, i*3 + 2); plt.imshow(masks[i].squeeze(), cmap='gray'); plt.title("Ground Truth"); plt.axis('off')
        plt.subplot(num_samples, 3, i*3 + 3); plt.imshow(img_to_show); plt.imshow(preds[i].squeeze(), cmap='jet', alpha=0.5); plt.title("Predicted Overlay"); plt.axis('off')
    
    plt.tight_layout(); plt.show()

def plot_eval_graphics(results_df):
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 2, 1); sns.boxplot(data=results_df[['Dice-Coefficient', 'IoU']], palette="viridis"); plt.title('Distribution of Segmentation Metrics'); plt.ylabel('Score')
    plt.subplot(2, 2, 2); sns.boxplot(data=results_df[['Hausdorff-Distance', 'ASD']], palette="plasma"); plt.title('Distribution of Boundary Error Metrics'); plt.ylabel('Pixel Distance')
    plt.tight_layout(); plt.show()



**MAIN FILE (BASE CLASS)**

Well,this is your PUBLIC STATIC VOID MAIN() 

In [ ]:
# Main function to run the entire thing.

import torch
import numpy as np
from config_pytorch import Config
from data_loader_pytorch import get_loaders
from model_pytorch import AttentionUNet
from train_pytorch import train_model
from evaluate_pytorch import evaluate_model, display_results_table
from visualize_pytorch import plot_learning_curves, visualize_predictions, plot_eval_graphics

def run_pipeline():
    """Executes the complete training and evaluation pipeline using PyTorch."""
    
    # 1. Configuration and Setup
    config = Config()
    np.random.seed(config.RANDOM_SEED)
    torch.manual_seed(config.RANDOM_SEED)
    torch.cuda.manual_seed(config.RANDOM_SEED)
    
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {DEVICE}")

    # 2. Prepare DataLoaders
    train_loader, val_loader, test_loader = get_loaders(config)
    
    # 3. Initialize Model
    model = AttentionUNet(n_channels=config.IMG_CHANNELS, n_classes=config.NUM_CLASSES).to(DEVICE)
    
    # 4. Train Model
    # Set to True to load a pre-trained model instead of training
    LOAD_PRETRAINED = False
    if LOAD_PRETRAINED:
        model.load_state_dict(torch.load(config.MODEL_SAVE_PATH))
        history = None # No history if not training
    else:
        model, history = train_model(config, model, train_loader, val_loader, DEVICE)

    # 5. Evaluate Model
    results_df = evaluate_model(model, test_loader, DEVICE)
    display_results_table(results_df)
    
    # 6. Visualize Results
    print("\n--- Generating Visualizations ---")
    if history:
        plot_learning_curves(history)
    visualize_predictions(model, test_loader, DEVICE)
    plot_eval_graphics(results_df)

if __name__ == '__main__':
    run_pipeline()
